<a href="https://colab.research.google.com/github/anuj0721/sign-language-to-text-translator/blob/main/basic_data_augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initialization

In [1]:
from google.colab import files
import os
import tensorflow

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

import matplotlib.pyplot as plt

In [ ]:
if not os.path.exists(os.path.expanduser('~')+'/.kaggle'):
    ! mkdir ~/.kaggle
os.chdir(os.path.expanduser('~')+'/.kaggle')
if not os.path.exists(os.path.expanduser('~')+'/.kaggle/kaggle.json'):
    kaggle_api_file = files.upload()
    ! kaggle datasets download -d grassknoted/asl-alphabet
    ! unzip -q asl-alphabet.zip
    ! rm -rf asl_alphabet_train/asl_alphabet_train/del

In [ ]:
! ls

# Data preprocessing and Data Augmentation

In [ ]:
train_data = 'asl_alphabet_train/asl_alphabet_train'

In [ ]:
image_gen = ImageDataGenerator(rotation_range=20,
                               width_shift_range=0.1,
                               height_shift_range=0.1,
                               brightness_range=(0.2,1),
                               shear_range=45,
                               zoom_range=(0.5,1.5),
                               channel_shift_range=100,
                               fill_mode='reflect',
                               horizontal_flip=True,
                               rescale=1/255,
                               validation_split=0.2)

In [ ]:
train_gen = image_gen.flow_from_directory(train_data,
                                          target_size=(224,224),
                                          class_mode='categorical',
                                          color_mode='rgb',
                                          shuffle=True,
                                          batch_size=32,
                                          seed=0,
                                          subset='training')

In [ ]:
val_gen = image_gen.flow_from_directory(train_data,
                                        target_size=(224,224),
                                        class_mode='categorical',
                                        color_mode='rgb',
                                        shuffle=True,
                                        batch_size=32,
                                        seed=0,
                                        subset='validation')

In [ ]:
train_gen.class_indices

# Creating model

In [ ]:
model = models.Sequential()

model.add(layers.Conv2D(filters=64,
                        kernel_size=9,
                        padding='same',
                        activation='relu',
                        input_shape=(224, 224, 3)))
model.add(layers.MaxPooling2D(pool_size=2,
                              padding='same'))
model.add(layers.Conv2D(filters=64,
                        kernel_size=9,
                        padding='same',
                        activation='relu'))
model.add(layers.MaxPooling2D(pool_size=2,
                              padding='same'))
model.add(layers.Conv2D(filters=64,
                        kernel_size=9,
                        padding='same',
                        activation='relu'))
model.add(layers.MaxPooling2D(pool_size=2,
                              padding='same'))
model.add(layers.Conv2D(filters=64,
                        kernel_size=9,
                        padding='same',
                        activation='relu'))
model.add(layers.MaxPooling2D(pool_size=2,
                              padding='same'))
model.add(layers.Flatten())
model.add(layers.Dense(units=128,
                        activation='relu'))
model.add(layers.Dense(units=128,
                        activation='relu'))
model.add(layers.Dense(units=128,
                        activation='relu'))
model.add(layers.Dense(units=128,
                        activation='relu'))
    
model.add(layers.Dense(28, activation='softmax'))

model.summary()

In [ ]:
model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
early_stopping_callback = EarlyStopping(monitor='val_loss',
                                        patience=5,
                                        mode='min',
                                        restore_best_weights=True)

# Training the model

In [ ]:
history = model.fit(train_gen,
                    epochs=60,
                    validation_data=val_gen,
                    callbacks=[early_stopping_callback])

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))
 
plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
 
plt.figure()
 
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
 
plt.show()

# Saving Model

In [ ]:
model.save('../asl_basic_data_augmentation.h5')